Rather than bootstrapping for eps, use MC methods

In [1]:
import gym
import numpy as np

In [27]:
print('Num actions: %i'%env.nA)

Num actions: 6


In [3]:
import Box2D
env = gym.make('Blackjack-v0')#.env
rand_pol = np.random.randint(0,6,1000)
obs= env.reset()
#env.render()

In [18]:
print(env.action_space)
print(env.observation_space)

Discrete(2)
Tuple(Discrete(32), Discrete(11), Discrete(2))


In [193]:
def policy(score):
    return 0 if score>=20 else 1

def every_visit_mc(env, n_eps=100):
    all_visits={}
    count_visits={}
    returns ={}
    
    #generate episode 
    for ep in range(n_eps):
        list_s=[]
        
        ep_visits={}
        ep_counts = {}
        ep_returns = {}
        
        #print('Episode %i'%ep)
        
        rew=0.0
        obs =env.reset()
        list_s.append([obs[0], rew])
        
        for t in range(100):
            #print('Observation: ',obs)
            score, dealer_score, ace = obs
            action = policy(score)
            #print('Action: %s'%action)
            
            obs, rew, done, info = env.step(action)
            #print('Reward', rew)
            list_s.append([obs[0], float(rew)])
            
            if done:
                #print('Final reward: ',float(rew))
                break
            #print("State: %s --> action %s, reward: %s, done: %s"%(obs[0], action, rew, done))
            

        #Find the FIRST time each state was visited in the episode 
        for (state, rew) in list_s:
            if not state in ep_visits.keys():
                if (state==20 or state==21) and rew==0.0:
                    continue
                else:
                    ep_visits[state]=float(rew)
                    ep_counts[state]=1.0
                    all_visits[state]=float(rew)
                    count_visits[state]=1.0
            else:
                ep_visits[state]+=float(rew)
                ep_counts[state]+=1.0
                count_visits[state]+=1.0
                all_visits[state]+=float(rew)
       
        for (key, val) in ep_visits.items():
            ep_returns[key]=ep_visits[key]/ep_counts[key]
        #print('Returns for this episode: %s \n'%ep_returns)

    #Take the average across all episodes 
    for (key, val) in all_visits.items():
        returns[key]=all_visits[key]/count_visits[key]
    #print('Final avg returns: %s \n'%returns)
        
    return returns
    
dic = every_visit_mc(env)
print(dic.values())
print(sorted(dic, lambda x:x[1]))

dict_values([0.0, -1.0, 0.0, -1.0, 1.0, 0.0, -1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, -1.0, 1.0, -1.0, -1.0, 0.0, 0.0, -1.0, 0.0, 0.0])


TypeError: must use keyword argument for key function

Becuase we cannot subtract cards in blackjack, every-visit MC would produce the same results as first-visit